# 布尔检索

## 1、词项文档矩阵

### 读取数据集

In [1]:
import os

docs = []
for filename in os.listdir("data"):
    filepath = os.path.join("data", filename)
    docs.append({
        "text": open(filepath, "r").read(),
        "name": filename
    })

In [2]:
os.listdir("data")

['tempest.txt',
 'juliuscaesar.txt',
 'othello.txt',
 'antonyandcleopatra.txt',
 'macbeth.txt',
 'hamlet.txt']

In [3]:
docs

[{'text': "\tTHE TEMPEST\n\n\n\tDRAMATIS PERSONAE\n\n\nALONSO\tKing of Naples.\n\nSEBASTIAN\this brother.\n\nPROSPERO\tthe right Duke of Milan.\n\nANTONIO\this brother, the usurping Duke of Milan.\n\nFERDINAND\tson to the King of Naples.\n\nGONZALO\tan honest old Counsellor.\n\n\nADRIAN\t|\n\t|  Lords.\nFRANCISCO\t|\n\n\nCALIBAN\ta savage and deformed Slave.\n\nTRINCULO\ta Jester.\n\nSTEPHANO\ta drunken Butler.\n\n\tMaster of a Ship. (Master:)\n\n\tBoatswain. (Boatswain:)\n\n\tMariners. (Mariners:)\n\nMIRANDA\tdaughter to Prospero.\n\nARIEL\tan airy Spirit.\n\n\nIRIS\t|\n\t|\nCERES\t|\n\t|\nJUNO\t|  presented by Spirits.\n\t|\nNymphs\t|\n\t|\nReapers\t|\n\n\n\tOther Spirits attending on Prospero.\n\n\nSCENE\tA ship at Sea: an island.\n\n\n\n\n\tTHE TEMPEST\n\n\nACT I\n\n\n\nSCENE I\tOn a ship at sea: a tempestuous noise\n\tof thunder and lightning heard.\n\n\n\t[Enter a Master and a Boatswain]\n\nMaster\tBoatswain!\n\nBoatswain\tHere, master: what cheer?\n\nMaster\tGood, speak to the m

### 分词

In [4]:
from nltk.tokenize import word_tokenize

tokenized_docs = []
for doc in docs:
    tokenized_docs.append({
        "words": set([word.lower() for word in word_tokenize(doc["text"])]),
        "name": doc["name"]
    })

In [5]:
tokenized_docs[0]

{'words': {'departing',
  'perchance',
  'throat',
  'unwillingly',
  'tutors',
  'hard',
  'darling',
  'frustrate',
  'gait',
  'humanely',
  'chick',
  'worship',
  'lied',
  'sometime',
  'sea-water',
  'drowsiness',
  'grew',
  "'gaitist",
  'hanged',
  'snores',
  'delight',
  'acquisition',
  'tailor',
  'lift',
  'fen',
  'harbour',
  'nook',
  'loss',
  'unfit',
  'donation',
  'discharge',
  'sycorax',
  'must',
  'fertile',
  'pease',
  'cankers',
  'jupiter',
  'less',
  'rate',
  'kind',
  'gold',
  'goose',
  'teeth',
  'yours',
  'supportable',
  'none',
  'dare',
  'grows',
  'print',
  'boded',
  'give',
  'end',
  'garners',
  'peace',
  'slumber',
  'skins',
  'severally',
  'plague',
  'itch',
  'leather',
  'retain',
  '!',
  'chains',
  'gave',
  'presently',
  'lie',
  "'widow",
  'summon',
  'mind',
  'grow',
  'being',
  'aught',
  'prepared',
  'entrails',
  'moon',
  'attach',
  'laughed',
  'rounded',
  'alas',
  'imposter',
  'prisoners',
  'dreams',
  'wea

### 构建词项文档矩阵

In [6]:
doc_ids = []
words = []

for doc in tokenized_docs:
    doc_ids.append(doc["name"])
    words += doc["words"]

words = list(set(words))

In [7]:
len(words), len(doc_ids)

(10581, 6)

In [8]:
import numpy as np

relation_matrix = np.zeros((len(words), len(doc_ids)))

In [9]:
for (i, word) in enumerate(words):
    for (j, _) in enumerate(doc_ids):
        if word in tokenized_docs[j]["words"]:
            relation_matrix[i, j] = 1

In [10]:
relation_matrix

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 1., 1., 1., 1.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.]])

###  完成查询：Brutus AND Caesar AND NOT Calpurnia

In [ ]:
words.index("brutus")

In [ ]:
relation_matrix[words.index("brutus")] 

In [ ]:
relation_matrix[words.index("caesar")] 

In [ ]:
1-relation_matrix[words.index("calpurnia")] 

In [ ]:
relation_matrix[words.index("brutus")]  *  relation_matrix[words.index("caesar")]  * (1-relation_matrix[words.index("calpurnia")] )

In [ ]:
doc_ids

In [ ]:
# mercy and NOT caesar

relation_matrix[words.index("mercy")]  *  (1-relation_matrix[words.index("caesar")])

### 稀疏度分析

In [11]:
not_zero_count = 0
total_count = len(words) * len(doc_ids)

for (i, word) in enumerate(words):
    for (j, _) in enumerate(doc_ids):
        if word in tokenized_docs[j]["words"]:
            not_zero_count += 1

In [12]:
not_zero_count

21723

In [13]:
total_count

63486

In [14]:
not_zero_count / total_count

0.3421699272280503

## 2、倒排索引

#### 实现倒排索引

In [15]:
inverted_index = {}


for doc in tokenized_docs:
    for word in doc["words"]:
        if word not in inverted_index:
            inverted_index[word] = []
        inverted_index[word].append(doc["name"])

In [16]:
inverted_index["caesar"]

['juliuscaesar.txt',
 'othello.txt',
 'antonyandcleopatra.txt',
 'macbeth.txt',
 'hamlet.txt']

In [17]:
inverted_index["calpurnia"]

['juliuscaesar.txt']

#### 倒排索引实现布尔查询

通过上述倒排索引，完成以下布尔查询的结果
- **Brutus AND Caesar AND (NOT Calpurnia)**

In [18]:
inverted_index["brutus"]

['juliuscaesar.txt', 'antonyandcleopatra.txt', 'hamlet.txt']

In [21]:
result = []
for t1 in inverted_index["brutus"]:
    if t1 in inverted_index["caesar"]:
        if t1 not in inverted_index["calpurnia"]:
            result.append(t1)

In [22]:
result

['antonyandcleopatra.txt', 'hamlet.txt']